<a href="https://colab.research.google.com/github/rlepsch/argus/blob/main/argus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <span style="color:blue">REINALDO LEPSCH NETO - TEST FOR ARGUS</span>


https://movile.blog/introducao-a-spark-usando-o-google-colab/

https://ichi.pro/pt/observacoes-sobre-como-salvar-dados-com-spark-3-0-148135666658161


## 2.1 Downloading wget



In [2]:
#instalando o módulo wget
%%capture
!pip install -q wget
#criando pasta para salvar as bases de dados
!mkdir data

## 2.2 Downloading the databases as copied to github



In [3]:
#baixando os dados das tabelas de dimensão
import wget

url = "https://raw.githubusercontent.com/rlepsch/argus/main/repo1.csv"
wget.download(url, "data/repo1.csv")

url = "https://raw.githubusercontent.com/rlepsch/argus/main/repo2.csv"
wget.download(url, "data/repo2.csv")

url = "https://raw.githubusercontent.com/rlepsch/argus/main/vhf_code.csv"
wget.download(url, "data/vhf_code.csv")

'data/vhf_code.csv'

# 3 Apache Spark Cluster

In [4]:
#instalando Java Runtime Environment (JRE) versão 8
%%capture
!apt-get remove openjdk*
!apt-get update --fix-missing
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [5]:
#baixando Apache Spark versão 3.0.0
%%capture
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-bin-hadoop2.7.tgz && rm spark-3.0.0-bin-hadoop2.7.tgz

In [6]:
import os
#configurando a variável de ambiente JAVA_HOME
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#configurando a variável de ambiente SPARK_HOME
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"

In [7]:
%%capture
#instalando o pacote findspark
!pip install -q findspark==1.4.2
#instalando o pacote pyspark
!pip install -q pyspark==3.0.0

In [8]:
#importando o módulo findspark
import findspark
#carregando a variávels SPARK_HOME na variável dinâmica PYTHONPATH
findspark.init()

In [9]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("pyspark-notebook").master("local[*]").getOrCreate()

# 4 Preparação dos Dados


In [10]:
#criando e exibindo o DataFrame para a tabela repo1

repo1 = spark.read.csv(path="data/repo1.csv", header=True, sep=",")
repo1.show(5)


+-----------------+-----------+--------------------+------+---------------------+-------------------+--------------+------------+
|vhf_repository_id|vhf_code_id|publication_datetime| value|applies_from_datetime|applies_to_datetime|forward_period|forward_year|
+-----------------+-----------+--------------------+------+---------------------+-------------------+--------------+------------+
|        184950377|      45747|    15/12/2021 13:23|     2|     15/12/2021 13:22|   15/12/2021 13:23|             2|        2022|
|        184950374|      45743|    15/12/2021 13:23|2.0775|     15/12/2021 13:22|   15/12/2021 13:23|             2|        2022|
|        184950371|      45745|    15/12/2021 13:23|2.0775|     15/12/2021 13:22|   15/12/2021 13:23|             2|        2022|
|        184950368|      45744|    15/12/2021 13:23|2.0776|     15/12/2021 13:22|   15/12/2021 13:23|             2|        2022|
|        184950365|      45746|    15/12/2021 13:23|2.0776|     15/12/2021 13:22|   15/12/

In [11]:
#criando e exibindo o DataFrame para a tabela repo2

repo2 = spark.read.csv(path="data/repo2.csv", header=True, sep=",")
repo2.show(5)


+-----------------+-----------+--------------------+-----+---------------------+-------------------+--------------+------------+
|vhf_repository_id|vhf_code_id|publication_datetime|value|applies_from_datetime|applies_to_datetime|forward_period|forward_year|
+-----------------+-----------+--------------------+-----+---------------------+-------------------+--------------+------------+
|        184931068|      45691| 15/12/2021 12:38:00|72.65|  15/12/2021 12:37:00|15/12/2021 12:38:00|             4|        2022|
|        184931061|      45687| 15/12/2021 12:38:00|12.00|  15/12/2021 12:37:00|15/12/2021 12:38:00|             3|        2022|
|        184931059|      45683| 15/12/2021 12:38:00|72.93|  15/12/2021 12:37:00|15/12/2021 12:38:00|             3|        2022|
|        184931057|      45685| 15/12/2021 12:38:00|72.91|  15/12/2021 12:37:00|15/12/2021 12:38:00|             3|        2022|
|        184931055|      45684| 15/12/2021 12:38:00|72.93|  15/12/2021 12:37:00|15/12/2021 12:38:

In [12]:
#criando e exibindo o DataFrame para a tabela vhf_code

vhf_code = spark.read.csv(path="data/vhf_code.csv", header=True, sep=",")
vhf_code.show(5)


+--------------------+--------------------+--------------------+------------------+-----------+
|         description|external_description|   description_short|external_query_str|vhf_code_id|
+--------------------+--------------------+--------------------+------------------+-----------+
|Brent month 1 1-m...|     Crude Oil Brent|Brent M1 1-min close|             CLOSE|      45678|
|Brent month 1 1-m...|     Crude Oil Brent| Brent M1 1-min high|              HIGH|      45679|
|Brent month 1 1-m...|     Crude Oil Brent|  Brent M1 1-min low|               LOW|      45680|
|Brent month 1 1-m...|     Crude Oil Brent| Brent M1 1-min open|              OPEN|      45681|
|Brent month 1 1-m...|     Crude Oil Brent|Brent M1 1-min vo...|            VOLUME|      45682|
+--------------------+--------------------+--------------------+------------------+-----------+
only showing top 5 rows



In [13]:
#criando as visões temporárias para as tabelas
repo1.createOrReplaceTempView("repo1")
repo2.createOrReplaceTempView("repo2")
vhf_code.createOrReplaceTempView("vhf_code")

In [14]:
vhf_code

DataFrame[description: string, external_description: string, description_short: string, external_query_str: string, vhf_code_id: string]

# 5 Execução de Consultas com Foco nas Operações OLAP

In [15]:
query = """
SELECT vhf_repository_id
FROM repo1 
where value > 2
"""

spark.sql(query).show()

+-----------------+
|vhf_repository_id|
+-----------------+
|        184950361|
|        184950325|
|        184950320|
|        184950315|
|        184950310|
|        184950305|
|        184950299|
|        184950293|
|        184950287|
|        184950281|
|        184950275|
|        184950268|
|        184950262|
|        184950256|
|        184950250|
|        184950244|
|        184950376|
|        184950360|
|        184950324|
|        184950319|
+-----------------+
only showing top 20 rows



In [16]:
( 
  repo1.write
  .mode('append') # or append
  #.partitionBy(col_name) # this is optional
  .bucketBy(5, 'forward_period') # n is number of buckets
  .sortBy('forward_period')
  #.format('parquet') # this is optional, parquet is default
  .option('path','file:///Users/reinaldolepsch/Downloads')
  .saveAsTable('tb_output')
)

In [18]:
df = spark.table('tb_output')

In [20]:
query = """
SELECT *
FROM tb_output 
"""

spark.sql(query).show()

+-----------------+-----------+--------------------+-----+---------------------+-------------------+--------------+------------+
|vhf_repository_id|vhf_code_id|publication_datetime|value|applies_from_datetime|applies_to_datetime|forward_period|forward_year|
+-----------------+-----------+--------------------+-----+---------------------+-------------------+--------------+------------+
|        184950144|      45692|    15/12/2021 13:22|   38|     15/12/2021 13:21|   15/12/2021 13:22|             4|        2022|
|        184950138|      45688|    15/12/2021 13:22|72.45|     15/12/2021 13:21|   15/12/2021 13:22|             4|        2022|
|        184950132|      45690|    15/12/2021 13:22|72.44|     15/12/2021 13:21|   15/12/2021 13:22|             4|        2022|
|        184950126|      45689|    15/12/2021 13:22|72.45|     15/12/2021 13:21|   15/12/2021 13:22|             4|        2022|
|        184950120|      45691|    15/12/2021 13:22|72.44|     15/12/2021 13:21|   15/12/2021 13:

# AS PROXIMAS CELULAS IR SUBSTITUINDO PELOS CODIGOS SOLICITADOS

## 5.2 Operações Drill-Down e Roll-Up

**Definição**: Analisam os dados considerando níveis progressivos de agregação.

- Drill-down: níveis de agregação progressivamente mais detalhados, ou de menor granularidade.
- Roll-up: níveis de agregação progressivamente menos detalhados, ou de maior granularidade.

Para ilustrar as operações de drill-down e roll-up, considere a consulta base definida a seguir.

**Consulta base:** Qual o valor gasto em salários por ano, considerando cada **semestre**?

In [ ]:
query = """
SELECT dataAno, dataSemestre, ROUND(SUM(salario),2) AS `Valor gasto em salários por semestre`
FROM data JOIN pagamento ON data.dataPK = pagamento.dataPK 
GROUP BY dataAno, dataSemestre
ORDER BY dataAno, dataSemestre
"""

spark.sql(query).show()

+-------+------------+------------------------------------+
|dataAno|dataSemestre|Valor gasto em salários por semestre|
+-------+------------+------------------------------------+
|   2016|           1|                          2221308.54|
|   2016|           2|                          2221308.54|
|   2017|           1|                           4887639.9|
|   2017|           2|                           4887639.9|
|   2018|           1|                           7467763.2|
|   2018|           2|                           7467763.2|
|   2019|           1|                          9283833.18|
|   2019|           2|                          9283833.18|
|   2020|           1|                          9283833.18|
|   2020|           2|                          9283833.18|
+-------+------------+------------------------------------+



**Exemplo de consulta drill-down:** Qual o valor gasto em salários por ano, considerando cada **trimestre**?

In [ ]:
query = """
SELECT dataAno, dataTrimestre, ROUND(SUM(salario),2) AS `Valor gasto em salários por trimestre`
FROM data JOIN pagamento ON (data.dataPK = pagamento.dataPK) 
GROUP BY dataAno, dataTrimestre
ORDER BY dataAno, dataTrimestre
"""

spark.sql(query).show()

+-------+-------------+-------------------------------------+
|dataAno|dataTrimestre|Valor gasto em salários por trimestre|
+-------+-------------+-------------------------------------+
|   2016|            1|                           1110654.27|
|   2016|            2|                           1110654.27|
|   2016|            3|                           1110654.27|
|   2016|            4|                           1110654.27|
|   2017|            1|                           2443819.95|
|   2017|            2|                           2443819.95|
|   2017|            3|                           2443819.95|
|   2017|            4|                           2443819.95|
|   2018|            1|                            3733881.6|
|   2018|            2|                            3733881.6|
|   2018|            3|                            3733881.6|
|   2018|            4|                            3733881.6|
|   2019|            1|                           4641916.59|
|   2019

**Exemplo de consulta roll-up:** Qual o valor gasto em salários por **ano**?

In [ ]:
query = """
SELECT dataAno, ROUND(SUM(salario),2) AS `Valor gasto em salários por ano`
FROM data JOIN pagamento ON (data.dataPK = pagamento.dataPK) 
GROUP BY dataAno
ORDER BY dataAno
"""

spark.sql(query).show()

+-------+-------------------------------+
|dataAno|Valor gasto em salários por ano|
+-------+-------------------------------+
|   2016|                     4442617.08|
|   2017|                      9775279.8|
|   2018|                   1.49355264E7|
|   2019|                  1.856766636E7|
|   2020|                  1.856766636E7|
+-------+-------------------------------+



## 5.3 Operação Pivot

**Definição:** Reorienta a visão multidimensional dos dados, oferecendo diferentes perspectivas dos mesmos dados.

Para ilustrar a operação pivot, considere a consulta base definida a seguir. 

**Consulta base:**  Qual o valor gasto em salários por ano, considerando cada nível de cargo?

In [ ]:
query = """
SELECT dataAno, cargoNivel, ROUND(SUM(salario),2) AS `Gastos em Salários`
FROM pagamento JOIN data ON pagamento.dataPK = data.dataPK 
               JOIN cargo ON pagamento.cargoPK = cargo.cargoPK 
GROUP BY dataAno, cargoNivel
ORDER BY dataAno, cargoNivel
"""

spark.sql(query).show()

+-------+----------+------------------+
|dataAno|cargoNivel|Gastos em Salários|
+-------+----------+------------------+
|   2016|    JUNIOR|         489456.84|
|   2016|     PLENO|        1454152.44|
|   2016|    SENIOR|         2499007.8|
|   2017|    JUNIOR|         1030642.8|
|   2017|     PLENO|        3791593.92|
|   2017|    SENIOR|        4953043.08|
|   2018|    JUNIOR|         1393282.2|
|   2018|     PLENO|        5357227.44|
|   2018|    SENIOR|        8185016.76|
|   2019|    JUNIOR|        1755714.36|
|   2019|     PLENO|        6132228.24|
|   2019|    SENIOR|     1.067972376E7|
|   2020|    JUNIOR|        1755714.36|
|   2020|     PLENO|        6132228.24|
|   2020|    SENIOR|     1.067972376E7|
+-------+----------+------------------+



**Exemplo de consulta pivot:** Qual o valor gasto em salários por nível de cargo, considerando cada ano?

In [ ]:
query = """
SELECT cargoNivel, dataAno, ROUND(SUM(salario),2) AS `Gastos em Salários`
FROM pagamento JOIN data ON pagamento.dataPK = data.dataPK 
               JOIN cargo ON pagamento.cargoPK = cargo.cargoPK 
GROUP BY cargoNivel, dataAno
ORDER BY cargoNivel, dataAno
"""

spark.sql(query).show()

+----------+-------+------------------+
|cargoNivel|dataAno|Gastos em Salários|
+----------+-------+------------------+
|    JUNIOR|   2016|         489456.84|
|    JUNIOR|   2017|         1030642.8|
|    JUNIOR|   2018|         1393282.2|
|    JUNIOR|   2019|        1755714.36|
|    JUNIOR|   2020|        1755714.36|
|     PLENO|   2016|        1454152.44|
|     PLENO|   2017|        3791593.92|
|     PLENO|   2018|        5357227.44|
|     PLENO|   2019|        6132228.24|
|     PLENO|   2020|        6132228.24|
|    SENIOR|   2016|         2499007.8|
|    SENIOR|   2017|        4953043.08|
|    SENIOR|   2018|        8185016.76|
|    SENIOR|   2019|     1.067972376E7|
|    SENIOR|   2020|     1.067972376E7|
+----------+-------+------------------+



## 5.4 Operação Drill-Across

**Definição:** Compara medidas numéricas de tabelas de fatos diferentes, utilizando pelo menos uma dimensão em comum. 

**Exemplo de consulta:** Qual o total gasto com salários e qual o total de receitas recebidas, considerando cada ano?

In [ ]:
# utilizando a cláusula JOIN ... ON ...
query = """
SELECT anoPag AS `Ano`, ROUND(salario,2) AS `Total Gasto com Salários`, ROUND(receita,2) AS `Total de Receitas Recebidas`
FROM ( SELECT dataAno, SUM(salario)  
       FROM pagamento JOIN data on data.dataPK = pagamento.dataPK
       GROUP BY dataAno
      ) AS pag(anoPag, salario)
     JOIN 
     ( SELECT dataAno, SUM(receita)
       FROM negociacao JOIN data ON data.dataPK = negociacao.dataPK
       GROUP BY dataAno
      ) AS neg(anoNeg, receita) 
     ON anoPag = anoNeg 
ORDER BY anoPag
"""

spark.sql(query).show()

+----+------------------------+---------------------------+
| Ano|Total Gasto com Salários|Total de Receitas Recebidas|
+----+------------------------+---------------------------+
|2016|              4442617.08|                 4614246.95|
|2017|               9775279.8|                 7200423.35|
|2018|            1.49355264E7|              1.159353965E7|
|2019|           1.856766636E7|               3.53533183E7|
|2020|           1.856766636E7|              3.022217595E7|
+----+------------------------+---------------------------+



In [ ]:
# utilizando a cláusula WHERE
query = """
SELECT anoPag AS `Ano`, ROUND(salario,2) AS `Total Gasto com Salários`, ROUND(receita,2) AS `Total de Receitas Recebidas`
FROM ( SELECT dataAno, SUM(salario) 
       FROM pagamento JOIN data on data.dataPK = pagamento.dataPK
       GROUP BY dataAno
      ) AS pag(anoPag, salario), 
     ( SELECT dataAno, SUM(receita)
       FROM negociacao JOIN data ON data.dataPK = negociacao.dataPK
       GROUP BY dataAno
      ) AS neg(anoNeg, receita) 
WHERE anoPag = anoNeg 
ORDER BY anoPag
"""

spark.sql(query).show()

+----+------------------------+---------------------------+
| Ano|Total Gasto com Salários|Total de Receitas Recebidas|
+----+------------------------+---------------------------+
|2016|              4442617.08|                 4614246.95|
|2017|               9775279.8|                 7200423.35|
|2018|            1.49355264E7|              1.159353965E7|
|2019|           1.856766636E7|               3.53533183E7|
|2020|           1.856766636E7|              3.022217595E7|
+----+------------------------+---------------------------+



## 5.5 Extensões ROLLUP, CUBE e GROUPING SETS 

**Definição:** Constrém vários níveis de agregação.

- ROLLUP: criação de subtotais para as combinações dos atributos da lista de agrupamento de acordo com a ordem desses atributos. São criados n+1 níveis de agregação, sendo n o número de atributos especificados na lista de agrupamento.

- CUBE: criação de subtotais para todas as combinações dos atributos da lista de agrupamento. São criados 2ˆn (2 elevado a n) níveis, sendo n o número de atributos especificados na lista de agrupamento.

- GROUPING SETS: criação de subtotais para quaisquer combinações de atributos de agrupamentos. É criada a quantidade de subtotais especificados na lista de níveis de agregação desejados. 



**Exemplo de consulta com ROLLUP:** Liste as agregações que podem ser geradas a partir da soma da receita por setor do cliente e por cidade do cliente, para totais de receita superiores a 3.000.000,00. Crie subtotais considerando a ordem dos atributos na lista de agrupamento.

In [ ]:
query = """
SELECT clienteSetor, clientecidade, ROUND(SUM(receita),2) AS `Total de Receitas`
FROM cliente JOIN negociacao ON cliente.clientePk = negociacao.clientePK
GROUP BY ROLLUP (clienteSetor, clienteCidade)
HAVING SUM(receita) > 3000000
ORDER BY clienteSetor, clienteCidade
"""

spark.sql(query).show()

+-------------------+--------------+-----------------+
|       clienteSetor| clienteCidade|Total de Receitas|
+-------------------+--------------+-----------------+
|               null|          null|     8.89837042E7|
|BEBIDAS E ALIMENTOS|          null|     3.54087859E7|
|BEBIDAS E ALIMENTOS|BELO HORIZONTE|        4206811.8|
|BEBIDAS E ALIMENTOS|       MARILIA|        4033680.5|
|BEBIDAS E ALIMENTOS|        RECIFE|        3899358.3|
|BEBIDAS E ALIMENTOS|RIO DE JANEIRO|       7351629.95|
|BEBIDAS E ALIMENTOS|     SAO PAULO|        6177945.5|
|BEBIDAS E ALIMENTOS|    UBERLANDIA|       3283351.55|
|            CREDITO|          null|        6621387.7|
|              SAUDE|          null|    1.831261245E7|
|              SAUDE|        MANAUS|       3329638.25|
|              SAUDE|     SAO PAULO|       3963055.35|
|         TECNOLOGIA|          null|      1.6568033E7|
|         TECNOLOGIA|     SAO PAULO|        9200352.8|
|          VESTUARIO|          null|    1.207288515E7|
|         

**Exemplo de consulta com GROUPING SETS com semântica de ROLLUP:** Liste todas as agregações que podem ser geradas a partir da soma da receita por setor do cliente e por cidade do cliente, para totais de receita superiores a 3.000.000,00. Crie subtotais considerando a ordem dos atributos na lista de agrupamento.

In [ ]:
query = """
SELECT clienteSetor, clientecidade, ROUND(SUM(receita),2) AS `Total de Receitas`
FROM cliente JOIN negociacao ON cliente.clientePk = negociacao.clientePK
GROUP BY GROUPING SETS ((clienteSetor, clienteCidade), (clienteSetor), ())
HAVING SUM(receita) > 3000000
ORDER BY clienteSetor, clienteCidade
"""

spark.sql(query).show()

+-------------------+--------------+-----------------+
|       clienteSetor| clienteCidade|Total de Receitas|
+-------------------+--------------+-----------------+
|               null|          null|     8.89837042E7|
|BEBIDAS E ALIMENTOS|          null|     3.54087859E7|
|BEBIDAS E ALIMENTOS|BELO HORIZONTE|        4206811.8|
|BEBIDAS E ALIMENTOS|       MARILIA|        4033680.5|
|BEBIDAS E ALIMENTOS|        RECIFE|        3899358.3|
|BEBIDAS E ALIMENTOS|RIO DE JANEIRO|       7351629.95|
|BEBIDAS E ALIMENTOS|     SAO PAULO|        6177945.5|
|BEBIDAS E ALIMENTOS|    UBERLANDIA|       3283351.55|
|            CREDITO|          null|        6621387.7|
|              SAUDE|          null|    1.831261245E7|
|              SAUDE|        MANAUS|       3329638.25|
|              SAUDE|     SAO PAULO|       3963055.35|
|         TECNOLOGIA|          null|      1.6568033E7|
|         TECNOLOGIA|     SAO PAULO|        9200352.8|
|          VESTUARIO|          null|    1.207288515E7|
|         

**Exemplo de consulta com CUBE:** Liste todas as agregações que podem ser geradas a partir da soma da receita por setor do cliente e por cidade do cliente, para totais de receita superiores a 3.000.000,00.

In [ ]:
query = """
SELECT clienteSetor, clientecidade, ROUND(SUM(receita),2) AS `Total de Receitas`
FROM cliente JOIN negociacao ON cliente.clientePk = negociacao.clientePK
GROUP BY CUBE (clienteSetor, clienteCidade)
HAVING SUM(receita) > 3000000
ORDER BY clienteSetor, clienteCidade
"""

spark.sql(query).show(40)

+-------------------+--------------+-----------------+
|       clienteSetor| clienteCidade|Total de Receitas|
+-------------------+--------------+-----------------+
|               null|          null|     8.89837042E7|
|               null|BELO HORIZONTE|       4379523.85|
|               null|  CAMPO GRANDE|       3422117.05|
|               null|      CURITIBA|       3697625.35|
|               null|     FORTALEZA|       3391233.25|
|               null|        MANAUS|       5234539.25|
|               null|       MARILIA|        7289146.1|
|               null|  PORTO ALEGRE|        4319625.7|
|               null|        RECIFE|       4717719.45|
|               null|RIO DE JANEIRO|    1.525596265E7|
|               null|    SAO CARLOS|       4192741.95|
|               null|     SAO PAULO|    2.464599965E7|
|               null|    UBERLANDIA|        4357595.2|
|BEBIDAS E ALIMENTOS|          null|     3.54087859E7|
|BEBIDAS E ALIMENTOS|BELO HORIZONTE|        4206811.8|
|BEBIDAS E

**Exemplo de consulta com GROUPING SETS com semântica de CUBE:** Liste todas as agregações que podem ser geradas a partir da soma da receita por setor do cliente e por cidade do cliente, para totais de receita superiores a 3.000.000,00. Crie subtotais considerando a ordem dos atributos na lista de agrupamento.

In [ ]:
query = """
SELECT clienteSetor, clientecidade, ROUND(SUM(receita),2) AS `Total de Receitas`
FROM cliente JOIN negociacao ON cliente.clientePk = negociacao.clientePK
GROUP BY GROUPING SETS ((clienteSetor, clienteCidade), (clienteSetor), (clienteCidade), ())
HAVING SUM(receita) > 3000000
ORDER BY clienteSetor, clienteCidade
"""

spark.sql(query).show(40)

+-------------------+--------------+-----------------+
|       clienteSetor| clienteCidade|Total de Receitas|
+-------------------+--------------+-----------------+
|               null|          null|     8.89837042E7|
|               null|BELO HORIZONTE|       4379523.85|
|               null|  CAMPO GRANDE|       3422117.05|
|               null|      CURITIBA|       3697625.35|
|               null|     FORTALEZA|       3391233.25|
|               null|        MANAUS|       5234539.25|
|               null|       MARILIA|        7289146.1|
|               null|  PORTO ALEGRE|        4319625.7|
|               null|        RECIFE|       4717719.45|
|               null|RIO DE JANEIRO|    1.525596265E7|
|               null|    SAO CARLOS|       4192741.95|
|               null|     SAO PAULO|    2.464599965E7|
|               null|    UBERLANDIA|        4357595.2|
|BEBIDAS E ALIMENTOS|          null|     3.54087859E7|
|BEBIDAS E ALIMENTOS|BELO HORIZONTE|        4206811.8|
|BEBIDAS E

# 6 Execução de Consultas com Foco na Tomada de Decisão
As consultas OLAP requisitadas por usuários de sistemas de suporte à decisão usualmente requerem que várias operações OLAP sejam realizadas simultaneamente. A seguir são ilustrados exemplos de consultas OLAP que podem ser requisitadas para a tomada de decisão estratégica.

## 6.1 Consulta 1

Qual é a média dos salários recebidos por nível do cargo e por sexo no ano de 2019?

Para se realizar esta consulta, é necessário obter dados das tabelas de dimensão `cargo`, `funcionario` e `data`, bem como da tabela de fatos `pagamento`. A junção estrela deve ocorrer considerando as seguintes integridades referenciais:
- `pagamento.cargoPK = cargo.cargoPK`
- `pagamento.funcPK = funcionario.funcPK`
- `pagamento.dataPK = data.dataPK` 

In [ ]:
# utilizando a cláusula JOIN ... ON ...
query = """
SELECT cargoNivel, funcSexo, ROUND(AVG(salario),2) AS `Média dos Salários`
FROM pagamento JOIN data ON data.dataPK = pagamento.dataPK
               JOIN cargo ON cargo.cargoPK = pagamento.cargoPK
               JOIN funcionario ON funcionario.funcPK = pagamento.funcPK 
WHERE dataAno = 2019
GROUP BY cargoNivel, funcSexo
ORDER BY cargoNivel, funcSexo
"""

spark.sql(query).show()

+----------+--------+------------------+
|cargoNivel|funcSexo|Média dos Salários|
+----------+--------+------------------+
|    JUNIOR|       F|           2440.23|
|    JUNIOR|       M|           2437.86|
|     PLENO|       F|           7641.94|
|     PLENO|       M|           6259.61|
|    SENIOR|       F|          12994.19|
|    SENIOR|       M|           14480.5|
+----------+--------+------------------+



In [ ]:
# utilizando a cláusula WHERE
query = """
SELECT cargoNivel, funcSexo, ROUND(AVG(salario),2) AS `Média dos Salários`
FROM pagamento, cargo, funcionario, data
WHERE data.dataPK = pagamento.dataPK
      AND cargo.cargoPK = pagamento.cargoPK
      AND funcionario.funcPK = pagamento.funcPK
      AND dataAno = 2019
GROUP BY cargoNivel, funcSexo
ORDER BY cargoNivel, funcSexo
"""

spark.sql(query).show()

+----------+--------+------------------+
|cargoNivel|funcSexo|Média dos Salários|
+----------+--------+------------------+
|    JUNIOR|       F|           2440.23|
|    JUNIOR|       M|           2437.86|
|     PLENO|       F|           7641.94|
|     PLENO|       M|           6259.61|
|    SENIOR|       F|          12994.19|
|    SENIOR|       M|           14480.5|
+----------+--------+------------------+



## 6.2 Consulta 2

Qual o total de gastos em salários considerando os estados nos quais as equipes estão localizadas no trimestre 3 do ano de 2020? 

Para se realizar esta consulta, é necessário obter dados das tabelas de dimensão `equipe` e `data`, bem como da tabela de fatos `pagamento`. A junção estrela deve ocorrer considerando as seguintes integridades referenciais:
- `pagamento.dataPK = data.dataPK`
- `pagamento.equipePK = equipe.equipePK`


In [ ]:
query = """
SELECT filialEstadoNome, ROUND(SUM(salario),2) AS Total
FROM pagamento JOIN data ON data.dataPK = pagamento.dataPK 
               JOIN equipe ON equipe.equipePK = pagamento.equipePK
WHERE dataAno = 2019
      AND dataTrimestre = 3
GROUP BY filialEstadoNome
ORDER BY Total 
"""

spark.sql(query).show()

+------------------+----------+
|  filialEstadoNome|     Total|
+------------------+----------+
|        PERNAMBUCO| 438121.26|
|MATO GROSSO DO SUL|1013857.74|
|    RIO DE JANEIRO|1258479.57|
|         SAO PAULO|1931458.02|
+------------------+----------+



## 6.3 Consulta 3

Qual o custo/benefício das equipes quando analisado o semestre 1 do ano de 2020?

A idea da consulta é relacionar os gastos em salários e os ganhos em receitas considerando cada equipe e o período especificado. Portanto, para se realizar essa consulta, é necessário obter dados das tabelas de dimensão `equipe` e `data`, bem como das tabelas de fatos `pagamento` e `negociacao`. 

 A junção estrela deve ocorrer considerando as seguintes integridades referenciais:
- `pagamento.dataPK = data.dataPK`
- `pagamento.equipePK = equipe.equipe.PK`
- `negociacao.dataPK = data.dataPK`
- `negociacao.equipePK = equipe.equipe.PK`


Uma observação muito importante refere-se ao fato que, para evitar dubiedade nas respostas, elas devem ser feitas sempre considerando a chave primária, desde que a chave primária identifica univocamente cada tupla. Depois de ser resolvida a consulta em termos da chave primária, então deve ser obtido os demais atributos a serem exibidos.

In [ ]:
query = """
-- obtendo os dados a serem exibidos na resposta
SELECT equipeNome, filialNome, ROUND(Lucro,2)
FROM equipe,
(
   SELECT pag.equipePK AS retornaEquipePK, (TotalReceita - TotalSalario) AS Lucro
   FROM ( 
        -- investigando os gastos em salarios de cada equipe no último semestre deste ano 
        SELECT equipePK, SUM(salario) AS TotalSalario
        FROM pagamento JOIN data ON data.dataPK = pagamento.dataPK
        WHERE dataSemestre = 1 
              AND dataAno = 2020
        GROUP BY equipePK 
        ORDER BY equipePK 
        ) AS pag,  
        (
        -- investigando os ganhos em receitas de cada equipe no último semestre deste ano   
        SELECT equipePK, SUM(receita) AS TotalReceita
        FROM negociacao JOIN data ON data.dataPK = negociacao.dataPK
        WHERE dataSemestre = 1 
              AND dataAno = 2020
        GROUP BY equipePK 
        ORDER BY equipePK
        ) AS neg
   WHERE pag.equipePK = neg.equipePK
   ) AS parte
WHERE equipe.equipePK = parte.retornaEquipePK
ORDER BY Lucro DESC

"""

spark.sql(query).show(5)

+--------------+--------------------+---------------+
|    equipeNome|          filialNome|round(Lucro, 2)|
+--------------+--------------------+---------------+
|BI & ANALYTICS|SAO PAULO - AV. P...|     4054890.24|
|BI & ANALYTICS|     RECIFE - CENTRO|     3788503.18|
| APP - DESKTOP|RIO DE JANEIRO - ...|      278013.26|
|  APP - MOBILE|SAO PAULO - AV. P...|      151160.49|
| APP - DESKTOP|SAO PAULO - AV. P...|       99579.85|
+--------------+--------------------+---------------+
only showing top 5 rows

